<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.12

     |████████████████████████████████| 83.1MB 367kB/s 
     |████████████████████████████████| 3.1MB 32.3MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [2]:
from __future__ import absolute_import, division, print_function
# 임포트 된 것을 파악하여 
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

print(tf.__version__)

1.12.0


In [0]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
# 4개 케이스, 소스
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]
# 4개 케이스, 타겟

In [4]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))

# s_vocab은 왜 정의하는가? 엠베딩 하기 위해 단어별를 원소로 공간을 형성
# sum은 문자열을 합치고 있다. 조인 함수의 경우 어떻게 되는가? 안됨
# 차원이 늘어나면 join으로 합칠 수 없나 봄. 그래서 sum을 사용하게 됨. 뒤의 []는 뭐지...; 변수의 type과 맞춰줘야 하나 봄
# set은 중복 제거, 고유 원소로만 만드나 봄. 얘는 형태가 set 타입으로 변경됨
# 그리고 다시 이걸 list로.. 형태를 복구시켜 줌. unique함수는 numpy나 pandas 쪽이어서 list엔 없음
# 핵심은 문장의 형태인 단어 조합을 해체하여 단어별로 취급하고 중복을 제거하는 것 

s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
# 거기에 패딩 원소 추가
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}
# 넘버링 딕셔너리

pprint(source2idx)


{'<pad>': 0,
 'I': 1,
 'a': 2,
 'changing': 3,
 'deep': 4,
 'difficult': 5,
 'fast': 6,
 'feel': 7,
 'for': 8,
 'framework': 9,
 'hungry': 10,
 'is': 11,
 'learning': 12,
 'tensorflow': 13,
 'very': 14}


In [5]:
# vocabularu for targets
t_vocab = list(set(sum(targets,[])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>','<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}
#타겟도 마찬가지 처리

pprint(target2idx)

{'<bos>': 1,
 '<eos>': 2,
 '<pad>': 0,
 '고프다': 3,
 '나는': 4,
 '딥러닝을': 5,
 '매우': 6,
 '배가': 7,
 '변화한다': 8,
 '빠르게': 9,
 '어렵다': 10,
 '위한': 11,
 '텐서플로우는': 12,
 '프레임워크이다': 13}


In [0]:
#전처리 함수 정의
def preprocess(sequences, max_len, dic, mode = 'source'):
  assert mode in ['source','target'], 'source와 target 중에 선택해주세요'
  # 뻑내면 나올 오류
  
  if mode == 'source': 
    # 소스 처리를 위한 셀(인코더)
    s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
    # 받는 것은 source로 문장 리스트를 받았음. 
    # 그런데 map 함수를 이용하면 리스트 내의 모든 하위 원소에 반응하는 것 같음. [문장1]이 아닌 ['단어1','단어2'] 등
    # dic은 뒤에서 source2idx 인덱스로 들어가는 것
    # 이 과정을 거쳐 s_input은 1차 embedding 완료
    s_len = list(map(lambda sentence : len(sentence), s_input))
    # s_input의 형태는 [['단어1_인덱스','단어2_인덱스'],[단어1인덱스, ..]]이럴건데..
    # 위와 차이는 for 문이 없다는 것이다. 
    # lambda 자체가 인스턴스 함수로 애시당초 시퀀스를 변수로 받는다. 위의 식은 거기에 한번 더 for를 씌워줘서 하윗단까지 가게 한거
    # 여기서는 투입 리스트 원소만 처리하면 되니 for 문을 쓰지 않았고 s_len은 투입받은 리스트의 원소(문장)의 길이를 세 준 것
    s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
    # 다시 s_input을 조정한다. pad_sequences는 패딩 씌우는 것
    # max_len에 맞게 10개의 길이로 통일시키기 위해 10개 이하는 뒤에 0(<pad>)을 붙여주고 넘치면 뒤를 잘라버림
    return s_len, s_input
    # 결과로 s_len과 s_input이 출력됨
    
  elif mode == 'target':
    # 타겟 처리를 위한 셀(디코더)
    # 투입물
    t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
    # 앞에 <bos>가 붙어서 문장의 시작을 알려서 y1이 문장의 첫 단어가 되도록 해준다. 
    # 뒤에 <eos>로 문장이 종료되었음을 선언
    # 이것은 번역이나 대화를 위한 세팅. 디코더가 만들 문장은 인코더 셀을 다 지난 후 bos로 시작하고 eos로 종료시킴
    t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
    # s_input과 비슷하다. <bos>,<eos>를     
    t_len = list(map(lambda sentence : len(sentence), t_input))
    t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
    
    # output
    t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
    t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
    t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
    
    return t_len, t_input, t_output
    

In [21]:
s_input = list(map(lambda sentence : [source2idx.get(token) for token in sentence], sources))
s_len = list(map(lambda sentence : len(sentence), s_input))
print(s_len)

[3, 4, 7, 5]


In [8]:
# 소스 전처리(위의 정의 함수)
s_max_len = 10
s_len, s_input =preprocess(sequences = sources,
                           max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[3, 4, 7, 5] [[ 1  7 10  0  0  0  0  0  0  0]
 [13 11 14  5  0  0  0  0  0  0]
 [13 11  2  9  8  4 12  0  0  0]
 [13 11 14  6  3  0  0  0  0  0]]


In [9]:
# 소스 전처리(위의 정의 함수)
s_max_len = 10
s_len, s_input =preprocess(sequences = sources,
                           max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[3, 4, 7, 5] [[ 1  7 10  0  0  0  0  0  0  0]
 [13 11 14  5  0  0  0  0  0  0]
 [13 11  2  9  8  4 12  0  0  0]
 [13 11 14  6  3  0  0  0  0  0]]


In [10]:
# 타겟 전처림
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 5, 6, 6] [[ 1  4  7  3  2  0  0  0  0  0  0  0]
 [ 1 12  6 10  2  0  0  0  0  0  0  0]
 [ 1 12  5 11 13  2  0  0  0  0  0  0]
 [ 1 12  6  9  8  2  0  0  0  0  0  0]] [[ 4  7  3  2  0  0  0  0  0  0  0  0]
 [12  6 10  2  0  0  0  0  0  0  0  0]
 [12  5 11 13  2  0  0  0  0  0  0  0]
 [12  6  9  8  2  0  0  0  0  0  0  0]]
